# Imports

In [1]:
import numpy as np
import pandas as pd
from pymongo import MongoClient

# Conexão com o MongoDB

In [2]:
client = MongoClient()
collec = client['portaisnoticias']['tech']

# Converter collection mongo para um DataFrame

In [4]:
df_dict = {
    '_id': [],
    'spider': [], 
    'url': [], 
    'titulo': [], 
    'qtd_comentarios': [],
    'autor': [],
    'revisor': [], 
    'data_publicacao': [], 
    'referencias': [],
    'tags': [],
    'conteudo_relacionado': [],
}

for doc in collec.find({}):
    for key, value in doc.items():
        df_dict[key].append(value)
        
    for missing in set(df_dict.keys()).difference(set(doc.keys())):
        df_dict[missing].append(None)
    

In [5]:
mongo_df = pd.DataFrame(df_dict)

In [6]:
mongo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138091 entries, 0 to 138090
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   _id                   138091 non-null  object        
 1   spider                138091 non-null  object        
 2   url                   138091 non-null  object        
 3   titulo                138090 non-null  object        
 4   qtd_comentarios       66748 non-null   float64       
 5   autor                 137280 non-null  object        
 6   revisor               29078 non-null   object        
 7   data_publicacao       138091 non-null  datetime64[ns]
 8   referencias           59660 non-null   object        
 9   tags                  137027 non-null  object        
 10  conteudo_relacionado  71959 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(9)
memory usage: 11.6+ MB


In [7]:
mongo_df.head()

,_id,spider,url,titulo,qtd_comentarios,autor,revisor,data_publicacao,referencias,tags,conteudo_relacionado
0,5e5461eeb27fbae055097ce5,olhardigital,https://olhardigital.com.br/noticia/vazamento-...,Vazamento indica sistema multitarefa do iPad p...,NaN,Luiz Nogueira,Cesar Schaeffer,2020-02-24 19:38:00,[https://www.cultofmac.com/686695/ios-14-multi...,"[iphone, apple, ipad, ios, iphone 11 pro max, ...",[https://olhardigital.com.br/noticia/apple-dev...
1,5e5461eeb27fbae055097ce6,olhardigital,https://olhardigital.com.br/carros-e-tecnologi...,Falta de baterias faz Audi interromper produçã...,NaN,Luiz Nogueira,Cesar Schaeffer,2020-02-24 17:20:00,[https://www.bloomberg.com/news/articles/2020-...,"[carro elétrico, tesla, audi, suv, veículo elé...",[https://olhardigital.com.br/noticia/primeiro-...
2,5e5461eeb27fbae055097ce7,olhardigital,https://olhardigital.com.br/noticia/essential-...,Essential Phone ganhará um gostinho do Android 11,NaN,Rafael Rigues,None,2020-02-24 17:49:00,[https://www.androidauthority.com/essential-ph...,"[android, smartphone, android 11]",[https://olhardigital.com.br/games-e-consoles/...
3,5e5461eeb27fbae055097ce8,olhardigital,https://olhardigital.com.br/cinema-e-streaming...,Netflix libera lista de 'Top 10' para identifi...,NaN,Cesar Schaeffer,None,2020-02-24 18:00:00,None,"[filmes, streaming, netflix, video, filmes onl...",[https://olhardigital.com.br/noticia/netflix-n...
4,5e5461eeb27fbae055097ce9,olhardigital,https://olhardigital.com.br/noticia/huawei-pre...,Huawei prepara lançamento de Smart Speaker na ...,NaN,Rafael Rigues,None,2020-02-24 16:25:00,[https://www.engadget.com/2020/02/24/huawei-so...,"[huawei, caixa de som, google home, alexa, cai...",[https://olhardigital.com.br/noticia/huawei-p4...


## Limpa caracteres errados no campo 'autor'

In [54]:
def remove_(value):
    if type(value) == str:
        if '\n' in value:
            value = value.replace('\n', '')
        
        value = value.strip()
    
    return value

In [55]:
mongo_df['autor'] = mongo_df['autor'].apply(remove_)

# Quantidade de noticias por site

In [8]:
mongo_df.groupby('spider')['titulo'].count()

spider
olhardigital    71343
tecmundo        66747
Name: titulo, dtype: int64

# Noticias por data de publicação

Fazer uma comparação ano/mês com a quantidade de publicações 

In [65]:
times = pd.DatetimeIndex(mongo_df['data_publicacao'])
noticias_ano = mongo_df.groupby([times.year, times.month])['titulo'].count()

noticias_ano.nlargest(10)

data_publicacao  data_publicacao
2016             3                  2215
2015             9                  2209
2016             2                  2142
2019             7                  2097
2017             8                  2089
2015             7                  2073
2019             10                 2073
2017             7                  2022
2015             8                  2016
2020             1                  2013
Name: titulo, dtype: int64

In [64]:
times = pd.DatetimeIndex(mongo_df['data_publicacao'])
noticias_ano = mongo_df.groupby([times.year, 'spider'])['titulo'].count()

noticias_ano.nlargest(10)

data_publicacao  spider      
2017             tecmundo        11938
2016             tecmundo        11833
2019             tecmundo        11118
2018             tecmundo        11000
2015             tecmundo        10962
2019             olhardigital    10332
2016             olhardigital     9273
2017             olhardigital     6965
2015             olhardigital     6933
2018             olhardigital     6300
Name: titulo, dtype: int64

# Quantidade de noticias por autor

In [63]:
mongo_df.groupby(['spider', 'autor'])['titulo'].count().nlargest(10)

spider        autor                
olhardigital  Redação Olhar Digital    10828
              Renato  Santino           7322
              Leonardo Pereira          6987
              Caroline Rocha            5170
tecmundo      Léo Müller                4580
              Felipe Payão              4565
olhardigital  Daniel Junqueira          4115
tecmundo      Nilton Kleina             4083
              Douglas Ciriaco           3646
olhardigital  Lucas Carvalho            3492
Name: titulo, dtype: int64

In [75]:
mongo_df.groupby(['spider', 'revisor', 'autor'])['spider'].count().nlargest(20)

spider        revisor           autor             
olhardigital  Igor Lopes        Daniel Junqueira      1126
              Marcelo Gripa     Caroline Rocha        1042
              Igor Lopes        Lucas Tavares          794
              Marcelo Gripa     Renato  Santino        793
              Leonardo Pereira  Caroline Rocha         762
              Igor Lopes        Caio Carvalho          700
              Marcelo Gripa     Gustavo Sumares        661
                                Kaluan Bernardo        652
              Leonardo Pereira  Gustavo Sumares        626
                                Renato  Santino        579
              Igor Lopes        Stephanie Kohn         570
              Leonardo Pereira  Juliana  AmÃ©rico      558
              Marcelo Gripa     Leonardo Pereira       518
                                Jeferson Goncalves     472
                                Juliana  AmÃ©rico      426
                                Daniel Junqueira       372
     

In [76]:
mongo_df.groupby(['spider', 'autor', 'revisor'])['spider'].count().nlargest(20)

spider        autor               revisor         
olhardigital  Daniel Junqueira    Igor Lopes          1126
              Caroline Rocha      Marcelo Gripa       1042
              Lucas Tavares       Igor Lopes           794
              Renato  Santino     Marcelo Gripa        793
              Caroline Rocha      Leonardo Pereira     762
              Caio Carvalho       Igor Lopes           700
              Gustavo Sumares     Marcelo Gripa        661
              Kaluan Bernardo     Marcelo Gripa        652
              Gustavo Sumares     Leonardo Pereira     626
              Renato  Santino     Leonardo Pereira     579
              Stephanie Kohn      Igor Lopes           570
              Juliana  AmÃ©rico   Leonardo Pereira     558
              Leonardo Pereira    Marcelo Gripa        518
              Jeferson Goncalves  Marcelo Gripa        472
              Juliana  AmÃ©rico   Marcelo Gripa        426
              Daniel Junqueira    Marcelo Gripa        372
     